# compare different BERT models for the tissue and cell type NER task
Fine tune existing bioNER models for biomedical tissue and cell type prediction

In [1]:
from datasets import Dataset, Features, Sequence, Value, ClassLabel
from transformers import AutoModelForTokenClassification, AutoTokenizer, DataCollatorForTokenClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import precision_recall_fscore_support
from iob_functions import *

import random

random.seed(6002)

In [2]:
BASE_DIR = "../data/"

training_f = process_tab_delim_iob(BASE_DIR + 'tags/fulltext_iob/fulltext_tissues_train.iob')
training_a = process_tab_delim_iob(BASE_DIR + 'tags/abstract_iob/abstract_tissues_train.iob')
training = {'sentences': training_f['sentences'] + training_a['sentences'], 'tags': training_f['tags'] + training_a['tags']}

valid_f = process_tab_delim_iob(BASE_DIR + 'tags/fulltext_iob/fulltext_tissues_validation.iob')
valid_a = process_tab_delim_iob(BASE_DIR + 'tags/abstract_iob/abstract_tissues_validation.iob')
validation = {'sentences': valid_f['sentences'] + valid_a['sentences'], 'tags': valid_f['tags'] + valid_a['tags']}

test_f = process_tab_delim_iob(BASE_DIR + 'tags/fulltext_iob/fulltext_tissues_test.iob')
test_a = process_tab_delim_iob(BASE_DIR + 'tags/abstract_iob/abstract_tissues_test.iob')
test = {'sentences': test_f['sentences'] + test_a['sentences'], 'tags': test_f['tags'] + test_a['tags']}

In [3]:
features = Features({"tokens": Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
                     "tags": Sequence(feature=ClassLabel(names=["O", "B-CELL_TYPE", "I-CELL_TYPE", "B-TISSUE", "I-TISSUE"]))})

In [4]:
training_ds = Dataset.from_dict({"tokens": training['sentences'], "tags": training['tags']}, features=features)
validation_ds = Dataset.from_dict({"tokens": validation['sentences'], "tags": validation['tags']}, features=features)
test_ds = Dataset.from_dict({"tokens": test['sentences'], "tags": test['tags']}, features=features)

In [5]:
all_tags = training_ds.features["tags"].feature
tag_list = training_ds.features["tags"].feature.names
id2tag = {idx: tag for idx, tag in enumerate(all_tags.names)}
tag2id = {tag: idx for idx, tag in enumerate(all_tags.names)}

In [6]:
# get the list of models to run
m_names = ['bert-base-uncased', 'google/electra-base-discriminator',
           'dmis-lab/biobert-base-cased-v1.2', 'bionlp/bluebert_pubmed_uncased_L-12_H-768_A-12',
           'kamalkraj/bioelectra-base-discriminator-pubmed',
           'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract']
MAX_LENGTH = 256

In [7]:
# from https://huggingface.co/docs/transformers/tasks/token_classification
def tokenize_and_align_labels(data, tknzr, max_length=50):
    tokenized_inputs = tknzr(data['tokens'], truncation=True, is_split_into_words=True, max_length=max_length)

    labels = []
    for i, label in enumerate(data['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [8]:
seqeval = evaluate.load("seqeval")

def flatten(l):
    return [item for sublist in l for item in sublist]
    
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [tag_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [tag_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    f1res = precision_recall_fscore_support(flatten(true_labels), flatten(true_predictions), labels=all_tags.names)

    df = list(zip(all_tags.names, f1res[2], f1res[0], f1res[1]))
    df = pd.DataFrame(df, columns = ['Level', 'F1-Score', 'Precision', 'Recall'])   
    print(df)

    return results

In [9]:
test_metrics = dict()

for m in m_names:
    print(m)
    tokenizer = AutoTokenizer.from_pretrained(m)
    # dynamically pad sentences to longest length in batch for efficiency
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

    train_tokenized = training_ds.map(tokenize_and_align_labels, batched=True, fn_kwargs={'tknzr': tokenizer, 'max_length': MAX_LENGTH})
    val_tokenized = validation_ds.map(tokenize_and_align_labels, batched=True, fn_kwargs={'tknzr': tokenizer, 'max_length': MAX_LENGTH})
    test_tokenized = test_ds.map(tokenize_and_align_labels, batched=True, fn_kwargs={'tknzr': tokenizer, 'max_length': MAX_LENGTH})

    model = AutoModelForTokenClassification.from_pretrained(
        m, num_labels=5, id2label=id2tag, label2id=tag2id
    )

    training_args = TrainingArguments(
        output_dir="model/" + m,
        learning_rate=2e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=3,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        push_to_hub=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized,
        eval_dataset=val_tokenized,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    test_preds = trainer.predict(test_tokenized)
    test_metrics[m] = test_preds.metrics


bert-base-uncased


Map:   0%|          | 0/20596 [00:00<?, ? examples/s]

Map:   0%|          | 0/3499 [00:00<?, ? examples/s]

Map:   0%|          | 0/5753 [00:00<?, ? examples/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/scratch/vy3/conda/envs/flambe/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/scratch/vy3/conda/envs/flambe/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all i

...

         Level  F1-Score  Precision    Recall
0            O  0.998637   0.999045  0.998228
1  B-CELL_TYPE  0.793201   0.758808  0.830861
2  I-CELL_TYPE  0.829493   0.814480  0.845070
3     B-TISSUE  0.875000   0.841871  0.910843
4     I-TISSUE  0.845238   0.887500  0.806818


/scratch/vy3/conda/envs/flambe/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


         Level  F1-Score  Precision    Recall
0            O  0.998662   0.998884  0.998439
1  B-CELL_TYPE  0.790765   0.769663  0.813056
2  I-CELL_TYPE  0.847458   0.875000  0.821596
3     B-TISSUE  0.879250   0.856164  0.903614
4     I-TISSUE  0.852459   0.821053  0.886364


/scratch/vy3/conda/envs/flambe/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


         Level  F1-Score  Precision    Recall
0            O  0.997243   0.998414  0.996075
1  B-CELL_TYPE  0.829135   0.782776  0.881331
2  I-CELL_TYPE  0.873091   0.843337  0.905022
3     B-TISSUE  0.892929   0.888442  0.897462
4     I-TISSUE  0.767624   0.659193  0.918750


In [10]:
test_metrics

{'bert-base-uncased': {'test_loss': 0.02012392319738865,
  'test_CELL_TYPE': {'precision': 0.7203182374541004,
   'recall': 0.8479827089337176,
   'f1': 0.7789543348775645,
   'number': 1388},
  'test_TISSUE': {'precision': 0.775096525096525,
   'recall': 0.8219037871033776,
   'f1': 0.7978142076502731,
   'number': 977},
  'test_overall_precision': 0.7415730337078652,
  'test_overall_recall': 0.8372093023255814,
  'test_overall_f1': 0.7864945382323734,
  'test_overall_accuracy': 0.9924709651581898,
  'test_runtime': 23.2553,
  'test_samples_per_second': 247.384,
  'test_steps_per_second': 30.961},
 'google/electra-base-discriminator': {'test_loss': 0.02117023430764675,
  'test_CELL_TYPE': {'precision': 0.7675033025099075,
   'recall': 0.8371757925072046,
   'f1': 0.8008270158511371,
   'number': 1388},
  'test_TISSUE': {'precision': 0.8149253731343283,
   'recall': 0.8382804503582395,
   'f1': 0.8264379414732594,
   'number': 977},
  'test_overall_precision': 0.7864231838030965,
  'te